In [1]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
import re
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster
from datetime import date as dt
import numpy as np

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))
pd.set_option('display.max_columns', None)

Uso de web scraping para tabular dados de sites de concurso público que agregam editais e combinação com geoprocessamento para criar um mapa interativo de localidades com editais abertos.

A tabulação dos dados é feita com a biblioteca Geopandas e operações com o shapefile da malha municipal disponibilizado pelo IBGE. O mapa interativo é construído com a biblioteca Folium

In [3]:
!pip install dbfread

In [4]:
!pip install descartes

In [5]:
!pip install folium

In [6]:
def load_geodata():

    brMun = gpd.read_file('BR_Municipios_2022.shp') #shapefile municipios                                       
    brMun['NM_MUN_tokens']=brMun['NM_MUN'].str.replace(' de ',' ').str\
                                                        .replace(' do ',' ').str\
                                                        .replace(' da ',' ').str\
                                                        .replace(' dos ',' ').str\
                                                        .replace(' das ',' ')
    return brMun

def pci_dataframe(url='https://www.pciconcursos.com.br/concursos/'):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
             '(KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36 Edg/129.0.0.0'}

    r = requests.get(url=url, headers=headers) #requesição ao pciconcursos
    content = soup(r.content, 'html.parser')

    classNa = content.body.div.next_element.next_element.findAll('div', {'class':'ca'})
    datas = [re.search('\d\d/\d\d/\d\d\d\d', x.findChild('div', {'class':'ce'}).text) for x in classNa]
    datas = pd.Series(
                [pd.to_datetime(x[0], format='%d/%m/%Y').date() 
                if x is not None else pd.NA for x in datas], name='data')
    titulos = [x.findChild().get('title') for x in classNa]
    links = pd.Series([x.findChild().get('href') for x in classNa], name='link')

    mask=[[(t.isupper()|t.istitle()) for t in titulo.split()] for titulo in titulos]
    tokens= [np.array(titulo.split(' '))[mask[n_]] for n_, titulo in enumerate(titulos)]

    dfPCI = pd.DataFrame({'cidade': np.nan*len(tokens), 
                          'estado': ''*len(tokens), 
                          'orgao': np.nan*len(tokens)},
                           index=range(0,len(tokens)))
    
    for n_, token in enumerate(tokens): #extraindo município, estado e órgão 
        municipio = []                  #do título da página
        for i_, t in enumerate(token):
            if t in ' '.join(brMun['NM_MUN_tokens'].values) and t!='Câmara':
                municipio.append(t)
                tokens[n_] = token.tolist().remove(t)
            elif t.strip(':') in brMun['UF'].values:
                dfPCI.loc[n_, 'estado'] = t.rstrip(':')
                tokens[n_] = token.tolist().remove(t)
            elif t.isupper() or t=='Câmara' or t=='Prefeitura':
                dfPCI.loc[n_, 'orgao'] = t.rstrip(':')
                tokens[n_] = token.tolist().remove(t)
        dfPCI.loc[n_, 'cidade'] = ' '.join(municipio)
    dfPCI = pd.concat([dfPCI, links, datas], axis=1)
    dfPCI['cidade'] = dfPCI['cidade'].replace('', np.nan)
    
    return dfPCI

Complementando com o estratégia concursos. Extrair dados de html pode requerer várias regras pela falta de padronização dos dados.

In [7]:
def estrat_dataframe(url='https://www.estrategiaconcursos.com.br/blog/concursos-abertos/'):
    url=url
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
             '(KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36 Edg/129.0.0.0'}

    r = requests.get(url=url, headers=headers)
    content = soup(r.content, 'html.parser')
    
    concursos = content.findAll(['h3'], {'class':"wp-block-heading"})
    detalhes = content.findAll('ul', {'class':"wp-block-list"})[2:-1]

    links = [i.find('a').get('href') for i in concursos]
    localidades = [i.text.split('(') for i in concursos]
    detalhes_t = [i.text.split('\n') for i in detalhes]
    
    for i, loc in enumerate(localidades):
        detalhes_t[i][0] = 'cidade: ' + loc[0]
        detalhes_t[i][-1] = 'estado: ' + loc[-1][:-1]
        detalhes_t[i].append(links[i])
    valores = []
    for info in detalhes_t:
        campo_valor = [i.split(':') if ':' in i else i.split(re.match(i, r'\xa0')) for i in info]
        valor = []
        for c_v in range(0,len(info), 1):
            valor.append(campo_valor[c_v][1].strip())
        valores.append(valor)
    
    dfEstrat = pd.DataFrame(valores[:-1])[[0, 6, 7, 3]]
    dfEstrat.columns = ['cidade', 'estado', 'link', 'data']
    
    dfEstrat['data'] = dfEstrat['data'].replace('[0-3][0-9] e |º|Enem|[0-9] e ' , '', regex=True)
    dfEstrat['data'] = dfEstrat['data'].apply(lambda x : pd.NA if 'suspen' in x else
                                             x+'/'+str(dt.today().year) if len(x)==5 else
                                          '0'+x+'/'+str(dt.today().year) if len(x)<5 else
                                          pd.NA if x.replace('/','').isnumeric()!=True
                                          else x)
    erros = []
    for n, data in enumerate(dfEstrat['data']):
        try:
            dfEstrat['data'][n] = pd.to_datetime(data)
        except:
            erros.append(n)
    
    dfEstrat = dfEstrat.drop(erros).reset_index(drop=True)
    dfEstrat['cidade'] = dfEstrat['cidade'].str.replace('Concurso', '').str.replace('Saúde', '').str.replace('Juiz', '')
    dfEstrat['estado'] = dfEstrat['estado'].str.replace(')', '')\
                                           .apply(lambda x : x if len(x)==2 and x.isupper() else '')

    municipios = brMun.sort_values('POP', ascending=False)['NM_MUN'].values
    for n, cid in enumerate(dfEstrat['cidade'].values):
        for mun in municipios:
            if mun in cid:
                dfEstrat.loc[n, 'cidade'] = mun
        
    dfEstrat.insert(2, 'orgao', dfEstrat['link'].apply(lambda x : 
                                                   'prefeitura' if 'prefeitura' in x else 
                                                   'câmara legislativa' if 'camara' in x else 'outros'))
    ufs = brMun['UF'].values
    for n, uf in enumerate(dfEstrat['cidade']):
        for uf_ in uf.split(' '):
            if uf_.isupper() and uf_ in ufs:
                dfEstrat.loc[n, 'estado'] = uf_
                dfEstrat.loc[n, 'cidade'] = np.nan
            elif (uf_.isupper() and uf_ not in ufs) or (uf_.istitle() and uf_ not in municipios):
                dfEstrat.loc[n, 'orgao'] = uf_
                dfEstrat.loc[n, 'cidade'] = np.nan
    return dfEstrat

def busca_localidade(df): #pegar cidade do endereço de um órgão sem localidade definida com o bing
    orgaos_sem_localidade = df[(df['cidade'].isna())&(df['orgao'].isna()==False)
                               &(df['orgao']!='Prefeitura')&(df['orgao']!='Câmara')].index
                               
    municipios = brMun.sort_values('POP', ascending=False)['NM_MUN'].values
    
    for n_, link in df.loc[orgaos_sem_localidade].iterrows():
        url = f'https://www.bing.com.br/search?q=endereço%{link["estado"].lower()}%{link["orgao"]}'
        headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                     '(KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36 Edg/129.0.0.0'}

        r = requests.get(url=url, headers=headers)
        content = soup(r.content, 'html.parser').text
        #content = ' '.join([c.text for c in content.findAll('em')])

        for cidade in municipios:
            try:
                if ' '+cidade in content:
                    df.loc[n_, 'cidade'] = cidade
                    print(cidade + '\n' + content.text)
                    break
                else:
                    pass
            except:
                if ' '+cidade in content:
                    df.loc[n_, 'cidade'] = cidade
                    print(cidade + '\n' + content)
                    break
                else:
                    pass
        print(lista[-1])

    return df

In [8]:
def busca_uf(df): #procura pelo estado que estejam faltando
    df.reset_index(drop=True, inplace=True)
    sem_uf = df[(df['estado'].isna())&(df['cidade'].isna()==False)]
    for n in sem_uf.index:
        for n_, cidade_ in enumerate(brMun['NM_MUN'].values):
            if df['cidade'][n] == cidade_:
                df.loc[n, 'estado'] = brMun.loc[n_, 'UF']
                print(n, cidade_, n_, brMun.loc[n_, 'UF'])
                
    return df

In [9]:
def remove_specialChars(df): #remover acentos e outros sinais gráficos das cidades
    global carac_sub
    carac_sub = {'Á':'A', 'À': 'A', 'Ã': 'A', 'Â': 'A',
                     'É': 'E', 'Ê': 'E', 'Í': 'I', 'Ó': 'O',
                     'Õ': 'O', 'Ô': 'O', 'Ú': 'U', 'Ç': 'C'}
    df['cidade'] = df['cidade'].str.upper().replace(carac_sub, regex=True).reset_index(drop=True)
    return df

In [10]:
def mergeData(df, data): #unir dados coletados com aqueles do shapefile do IBGE
    df = df.merge(data, left_on=['cidade','estado'], 
                        right_on=['NM_MUN','UF'], 
                        how='left').dropna()
    
    to_drop = df.columns[['_x' in x for x in df.columns]]
    df = gpd.GeoDataFrame(df, geometry='geometry').drop(to_drop, axis=1).reset_index(drop=True)
    df.drop(['NM_MUN','UF'],axis=1, inplace=True)
    
    return df

In [11]:
def addFields(df): #campos que serão utilizados na plotagem dos dados: latitude, longitude,
                    #centroid dos municipios e texto que será mostrado no popup
    df['links_ac'] = [f'<h4 style="text-align:center">{x[0]}</h4><br><a href={x[1]}'
                      f' target="_blank">{x[1]}</a>' for x in zip(df['cidade'], df["link"])]

    df['centroid'] = [x.centroid if x is not None else pd.NA for x in df['geometry']]
    df['centroid'] = df['geometry'].astype('geometry')
    df['y'], df['x'] = df.centroid.y, df.centroid.x

    return df

In [12]:
def plot_data(df): #plotar dados em mapa de cluster
    latitudeMedia = df[df['data'].isna()==False].centroid.y.median()
    longetudeMedia = df[df['data'].isna()==False].centroid.x.median()
    fmap = folium.Map(location=[latitudeMedia, longetudeMedia], zoom_start=7)
    mc = MarkerCluster(df[['y','x']], popups=df['links_ac'].tolist())
    for _, r in df.iterrows():

        sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j,
                               style_function=lambda x: {'fillColor': 'orange',
                                                          'weight': 1})
        geo_j.add_to(fmap)


    return fmap.add_child(mc)

In [13]:
brMun = load_geodata()

In [14]:
df = pd.concat([pci_dataframe(), estrat_dataframe()]).reset_index(drop=True)

C:\Users\Elizabeth\AppData\Local\Temp\ipykernel_3592\2021036300.py:40: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  dfEstrat['data'][n] = pd.to_datetime(data)
C:\Users\Elizabeth\AppData\Local\Temp\ipykernel_3592\2021036300.py:46: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dfEstrat['estado'] = dfEstrat['estado'].str.replace(')', '')\


In [15]:
#busca_localidade(df)

#df.to_csv('dados_tabulados.csv', index=False)
df=pd.read_csv('dados_tabulados.csv')

In [16]:
busca_uf(df)

,cidade,estado,orgao,link,data
0,NaN,,INSA,https://www.pciconcursos.com.br/noticias/insa-...,2024-12-26
1,NaN,,TRF,https://www.pciconcursos.com.br/noticias/trf-6...,2024-11-08
2,Osasco,SP,FITO,https://www.pciconcursos.com.br/noticias/fito-...,2024-11-18
3,Botucatu,SP,Prefeitura,https://www.pciconcursos.com.br/noticias/concu...,2024-11-17
4,Guarulhos,SP,Prefeitura,https://www.pciconcursos.com.br/noticias/prefe...,2024-11-21
...,...,...,...,...,...
486,NaN,,INSA,//www.estrategiaconcursos.com.br/blog/concurso...,2025-02-23 00:00:00
487,NaN,,INPP,//www.estrategiaconcursos.com.br/blog/concurso...,2025-02-02 00:00:00
488,Jardim,RJ,outros,//www.estrategiaconcursos.com.br/blog/concurso...,NaT
489,NaN,MG,Ativos,//www.estrategiaconcursos.com.br/blog/concurso...,2025-09-02 00:00:00


In [ ]:
mergeData(df, brMun).pipe(addFields).pipe(plot_data)

C:\Users\Elizabeth\AppData\Local\Temp\ipykernel_3592\3537775827.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['y'], df['x'] = df.centroid.y, df.centroid.x
C:\Users\Elizabeth\AppData\Local\Temp\ipykernel_3592\1514991943.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  latitudeMedia = df[df['data'].isna()==False].centroid.y.median()
C:\Users\Elizabeth\AppData\Local\Temp\ipykernel_3592\1514991943.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  longetudeMedia = df[df['data'].isna()==False].centroid.x.median()


In [ ]:
url = f'https://www.bing.com.br/search?q=endereço%{"guaraiprev"}%{"sp"}'
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
             '(KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36 Edg/129.0.0.0'}

r = requests.get(url=url, headers=headers)
content = soup(r.content, 'html.parser')